# API - Airports / Flights

## 1. Retrieve city data from MySQL database

In [ ]:
# import libraries
import pandas as pd
import requests
from sqlalchemy import create_engine, text
import os
from dotenv import load_dotenv
load_dotenv()
from datetime import datetime, timedelta
from pytz import timezone

In [ ]:
# Connection setup MySQL
schema = "city_infos"
host = "127.0.0.1"
user = "root"
password = os.getenv("MYSQL_PASSWORD") # password = "YOUR_PASSWORD"
port = 3306

# Create connection string for MySQL
connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

# Connection for API
rapidapi_key = os.getenv("RapidAPI-Key")    # rapidapi_key = "YOUR_RAPIDAPI_KEY"

In [ ]:
cities_df = pd.read_sql('cities', con=connection_string)
cities_df[['latitude','longitude']] = cities_df[['latitude','longitude']].round(2)
cities_df

## 2. API call for airport data

### Explore response for one city

In [ ]:
# pick latitude, longitude for Berlin example
lat = cities_df.loc[0,'latitude']
lon = cities_df.loc[0,'longitude']

url = "https://aerodatabox.p.rapidapi.com/airports/search/location"

headers = {
    	"X-RapidAPI-Key": rapidapi_key,
        "x-rapidapi-host": "aerodatabox.p.rapidapi.com"
}

querystring = {
    "lat": f"{lat}",
    "lon": f"{lon}",
    "radiusKm": "50",
    "limit": "10",
    "withFlightInfoOnly": "true"
}

berlin_response = requests.get(url, headers=headers, params=querystring)

berlin_response_json = berlin_response.json()
berlin_response_json

In [ ]:
berlin_response_json.keys() 

In [ ]:
pd.json_normalize(berlin_response_json['items'])

### Create function for multiple cities

In [ ]:
# define function for airport data for multiple cities

import pandas as pd
import requests
from dotenv import load_dotenv
import os
load_dotenv()

# Connection for API
rapidapi_key = os.getenv("RapidAPI-Key")    # rapidapi_key = "YOUR_RAPIDAPI_KEY"

def get_airports(cities_df):
    url = "https://aerodatabox.p.rapidapi.com/airports/search/location"

    headers = {
            "X-RapidAPI-Key": rapidapi_key,
            "x-rapidapi-host": "aerodatabox.p.rapidapi.com"
    }

    all_results =[]

    for i, row in cities_df.iterrows():
        lat = row['latitude']
        lon = row['longitude']
        city_id = row['city_id']

        querystring = {
            "lat": str(lat),
            "lon": str(lon),
            "radiusKm": "50",
            "limit": "10",
            "withFlightInfoOnly": "true"
        }
        response = requests.get(url, headers=headers, params=querystring)
        if response.status_code == 200:
            data = response.json()
            city_results = data.get("items", [])

            for airport in city_results:     # Add city_id to each airport
                airport['city_id'] = city_id
                all_results.append(airport)
        else:
            print(f"Error {response.status_code} at {row['id']}") # error handling

    airports_df = pd.DataFrame(all_results)
    airports_df = airports_df[['city_id','name','iata','icao']] # select the info needed
    airports_df = airports_df.rename(columns={'name':'airport_name'})

    return airports_df

In [ ]:
# call the function with cities_df
airports_df = get_airports(cities_df)
airports_df

In [ ]:
airports_df.info()

### Send data to MySQL airports table

In [ ]:
airports_df.to_sql('airports',
                 if_exists='append',
                 index=False,
                 con=connection_string
)

## 3. API call for flight data

In [ ]:
# Retrieve airport_df from MySQL database
airports_df = pd.read_sql("SELECT * FROM airports", con=connection_string)
airports_df

### Explore response for one airport & one time frame

In [ ]:
# retrieve flight data for Berlin airport from AeroDataBox API

# request for 00:00-11:59 o'clock of the next day (only 12 hour range possible)

icao = "EDDB"    # using icao code for the airbort
tomorrow = datetime.now() + timedelta(days=1)
date = tomorrow.strftime('%Y-%m-%d')  # date of the next day
time_1 = "00:00"
time_2 = "11:59"

url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{icao}/{date}T{time_1}/{date}T{time_2}"

# setting query parameters
querystring = {"withLeg":"true","direction":"Arrival","withCancelled":"false","withCodeshared":"true","withCargo":"false","withPrivate":"false","withLocation":"false"}

headers = {
    "X-RapidAPI-Key": rapidapi_key,
    "x-rapidapi-host": "aerodatabox.p.rapidapi.com"
}

response1 = requests.get(url, headers=headers, params=querystring)

In [ ]:
# check keys for example response1
flights_json_1 = response1.json()
flights_json_1.keys()

In [ ]:
#flights_json_1['arrivals']

In [ ]:
#  noamalize the table
normalized_table_1 = pd.json_normalize(flights_json_1['arrivals'])

In [ ]:
normalized_table_1.columns

In [ ]:
flights_json_1["arrivals"][0].keys()

In [ ]:
# extract relevant data for response1 for icao 'EDDB'
icao = "EDDB"
flight_items = []

for item in flights_json_1.get("arrivals", []): # Safely get 'arrivals'
    flight_item = {
        "arrival_airport_icao": icao,   # for example icao 'EDDB'
        "departure_airport_icao": item["departure"]["airport"].get("icao", None),
        "scheduled_arrival_time": item["arrival"]["scheduledTime"].get("local", None),
        "revised_arrival_time": item["arrival"]["revisedTime"].get("local", None), # 'revised time' could be empty
        "arrival_terminal": item["arrival"].get("terminal", None),
        "arrival_gate": item["arrival"].get("gate", None),
        "flight_number": item.get("number", None)
    }

    flight_items.append(flight_item)
    flights1_df = pd.DataFrame(flight_items)

flights1_df.head()

In [ ]:
# remove UTC for times
flights1_df["scheduled_arrival_time"] = flights1_df["scheduled_arrival_time"].str[:-6]
flights1_df["revised_arrival_time"] = flights1_df["revised_arrival_time"].str[:-6]
flights1_df.head()

### Create function for multiple airports and defined time frame

In [ ]:
# define function for retrieval of flight data for airports_df for the defined next days from now

# import libraries
import pandas as pd
import requests
from sqlalchemy import create_engine, text
import os
from dotenv import load_dotenv
load_dotenv()
from datetime import datetime, timedelta
from pytz import timezone
import time as _time

# Connection setup MySQL
schema = "city_infos"
host = "127.0.0.1"
user = "root"
password = os.getenv("MYSQL_PASSWORD") # password = "YOUR_PASSWORD"
port = 3306

# Create connection string for MySQL
connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

# Connection for API
rapidapi_key = os.getenv("RapidAPI-Key")    # rapidapi_key = "YOUR_RAPIDAPI_KEY"


def get_flight_data(airports_df, days=2): # set how many days ahead to pull data (incl. today)

    # define timezone and current local time
    tz_name='Europe/Berlin'
    local_tz = timezone(tz_name)
    now_local = datetime.now(local_tz)

    # build time windows with defined days and current local time
    intervals = []

    for d in range(days):    # iterate over the days
        day_date = now_local.date() + timedelta(days=d)
        day_str = day_date.strftime('%Y-%m-%d')

        if d == 0:  # today - start from current time
            
            if now_local.hour < 12:     # current time before noon
                intervals.append((      # first interval
                    f'{day_str}T{now_local.strftime('%H:%M')}',
                    f'{day_str}T11:59'
                ))
                intervals.append((f'{day_str})T12:00', f'{day_str}T23:59')) # second interval
            else:
                intervals.append((  # current time after noon
                    f'{day_str}T{now_local.strftime('%H:%M')}',
                    f'{day_str}T23:59'
                ))
        else:   # next days (full days)
            intervals.append((f"{day_str}T00:00", f"{day_str}T11:59"))
            intervals.append((f"{day_str}T12:00", f"{day_str}T23:59"))

    # define sleep time between requests
    sleep_s=1


    flight_items = []

    for i, row in airports_df.iterrows():   # iteration over airport_df icao codes

        for start_time, end_time in intervals:  # iteration over the time windows (list of tubles)
            url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{row['icao']}/{start_time}/{end_time}"
        
            querystring = {
                "withLeg": "true",
                "direction": "Arrival",
                "withCancelled": "false",
                "withCodeshared": "true",
                "withCargo": "false",
                "withPrivate": "false",
                "withLocation":"false"
            }
            headers = {
                'x-rapidapi-host': "aerodatabox.p.rapidapi.com",
                'x-rapidapi-key': rapidapi_key
            }
            response = requests.get(url, headers=headers, params=querystring)
        
            if response.status_code == 200:
                flights_json = response.json()
                
                # add retrieval_time for flight data
                retrieval_time = datetime.now(local_tz).strftime("%Y-%m-%d %H:%M:%S")
            
                for item in flights_json.get("arrivals", []):  # Safely get 'arrivals'
                    flight_item = {
                        "arrival_airport_icao": row['icao'],
                        "departure_airport_icao": item["departure"]["airport"].get("icao", None),
                        "scheduled_arrival_time": item["arrival"]["scheduledTime"].get("local", None),
                        "revised_arrival_time": item["arrival"].get("revisedTime", {}).get("local", None), # 'revised time' could be empty                                 
                        "arrival_terminal": item["arrival"].get("terminal", None),
                        "arrival_gate": item["arrival"].get("gate", None),
                        "flight_number": item.get("number", None),
                        "retrieval_time": retrieval_time
                    }
                    flight_items.append(flight_item)
            
            elif response.status_code == 204: # no content for this interval -> skip
                pass

            else:
                print(f"Error for {row['icao']} at {start_time}→{end_time}: {response.status_code}") # error handling

            _time.sleep(sleep_s) # sleep time between requests

    flights_df = pd.DataFrame(flight_items)

    # Ensure proper datetime format
    flights_df["scheduled_arrival_time"] = flights_df["scheduled_arrival_time"].str[:-6] # time without UTC
    flights_df["scheduled_arrival_time"] = pd.to_datetime(flights_df["scheduled_arrival_time"]) # convert to datetime format
    flights_df["revised_arrival_time"] = flights_df["revised_arrival_time"].str[:-6] 
    flights_df["revised_arrival_time"] = pd.to_datetime(flights_df["revised_arrival_time"])
    flights_df["retrieval_time"] = pd.to_datetime(flights_df["retrieval_time"]) # convert retrieval time to datetime format

    return flights_df

In [ ]:
# call flight function for time range until tomorrow
flights_df = get_flight_data(airports_df, days=2)

In [ ]:
flights_df_cl = flights_df.drop_duplicates()
flights_df_cl

In [ ]:
flights_df_cl.info()

### Send data to MySQL flights table

In [ ]:
flights_df_cl.to_sql('flights',
                 if_exists='append',
                 index=False,
                 con=connection_string)